In [2]:
import sys
from datetime import datetime
import pandas as pd 
import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.express as px
from scipy.stats import lognorm
from statsmodels.distributions.empirical_distribution import ECDF


In [3]:
print(sys.path)

['/home/ivan/projects/VPIN', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/ivan/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages']


In [4]:

from binance.client import Client
def get_data_from_binance(symbol='BTCUSDT', resample_freq='1h', start_time="1 Jan, 1970", end_time=None, futures=True):
    """
    從 Binance 上抓取數據，並存成 csv 檔案。
    Args:
        symbol (str, optional): 要抓取的交易對. Defaults to 'BTCUSDT'.
        resample_freq (str, optional): 要抓取的時間週期. Defaults to '1h'.
        start_time (str, optional): 要抓取的起始時間. Defaults to '1 Jan, 1970'.
        end_time (str, optional): 要抓取的結束時間. Defaults to None.
    Returns:
        df: 抓取的數據
    """
    # Binance API Token
    api_key = 'AccZsoScBxoX6xXZpVZD22iGx2BOFD35L0aIHB4LONPCHlx7A6Bd9gvFBS4WlYJG'
    api_secret = 'CG6RMwPrEn0UEHxRf6ThisxBTa8Og9TpoHbV4XuD0dhniyCo7MRBIfBE0WjbAq2I'
    client = Client(api_key, api_secret)  # 注意 futures=True
    # Fetch BTC/USDT 1hr data
    if futures:
        print('Fetching futures data...')
        klines = client.futures_historical_klines(symbol, resample_freq, start_time, end_time)
        if len(klines) == 0:
            print('Future data empty, fetching spot data...')
            klines = client.get_historical_klines(symbol, resample_freq, start_time, end_time)
    else:
        print('Fetching spot data...')
        try:
            klines = client.get_historical_klines(symbol, resample_freq, start_time, end_time)
            if len(klines) == 0:
                print('Spot data empty, fetching future data...')
                klines = client.futures_historical_klines(symbol, resample_freq, start_time, end_time)
        except:
            print('Spot data return error, fetching futures data...')
            klines = client.futures_historical_klines(symbol, resample_freq, start_time, end_time)
    # turn to df
    df = pd.DataFrame(klines, columns=['open_time', 'open', 'high', 'low', 'close', 'volume',
                      'close_time', 'quote_asset_volume', 'trades', 'taker_buy_volume', 'taker_buy_quote', 'ignored'])
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
    # choose interested columns
    df = df[['open_time', 'open', 'high', 'low', 'close', 'volume','taker_buy_volume']]
    # turn to float
    df[['open', 'high', 'low', 'close', 'volume', 'taker_buy_volume']] = df[[
        'open', 'high', 'low', 'close', 'volume', 'taker_buy_volume']].astype(float)

    df['taker_sell_volume'] = df['volume'] - df['taker_buy_volume']
    # # save csv
    # df.to_csv(folder_path + f'{symbol}_{resample_freq}.csv', index=False)
    # # print message
    # print('Data saved to ' + folder_path + f'{symbol}_{resample_freq}.csv')
    return df


In [5]:
symbols = ['BTCUSDT']
df_kbar = {}

for symbol in symbols : 
    print("Now Downloading : " + symbol)
    df_kbar[symbol] = get_data_from_binance(symbol= symbol, 
                        resample_freq='1m', 
                        start_time="1 Jan, 2017", 
                        end_time= None, 
                        futures=False)

Now Downloading : BTCUSDT
Fetching spot data...


In [5]:
symbols = ['ETHUSDT']
df_kbar = {}

for symbol in symbols : 
    print("Now Downloading : " + symbol)
    df_kbar[symbol] = get_data_from_binance(symbol= symbol, 
                        resample_freq='1m', 
                        start_time="1 Jan, 2017", 
                        end_time= None, 
                        futures=False)
df_kbar['ETHUSDT'].to_csv('eth.csv')

Now Downloading : ETHUSDT
Fetching spot data...


In [6]:
# df_kbar['SOLUSDT'].to_csv('sol.csv')
# df_kbar['ETHUSDT'].to_csv('eth.csv')
df_kbar['BTCUSDT'].to_csv('btc.csv')